In [1]:
!pip install -q llama-index
! pip install -q deeplake langchain pypdf
! pip install -q tensorflow_text  deeplake


In [34]:
!pip install -q google-cloud-aiplatform
!pip install -q  trulens-eval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.5/623.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.4/518.4 kB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 

# Lesson 1: Advanced RAG Pipeline

In [3]:


import os



In [4]:
!wget "https://wordpress.deeplearning.ai/wp-content/uploads/2022/10/eBook-How-to-Build-a-Career-in-AI.pdf"


--2023-12-02 05:44:39--  https://wordpress.deeplearning.ai/wp-content/uploads/2022/10/eBook-How-to-Build-a-Career-in-AI.pdf
Resolving wordpress.deeplearning.ai (wordpress.deeplearning.ai)... 164.92.64.128
Connecting to wordpress.deeplearning.ai (wordpress.deeplearning.ai)|164.92.64.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3717673 (3.5M) [application/pdf]
Saving to: ‘eBook-How-to-Build-a-Career-in-AI.pdf.1’

eBook-How-to-Build- 100%[===================>]   3.54M  --.-KB/s    in 0.1s    

2023-12-02 05:44:40 (28.3 MB/s) - ‘eBook-How-to-Build-a-Career-in-AI.pdf.1’ saved [3717673/3717673]



In [5]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 87a784c3-8e6f-463a-bdf3-5c8db4cb65db
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [7]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [8]:
from google.colab import auth

auth.authenticate_user()

In [24]:
import vertexai
from langchain.llms import VertexAI

PROJECT_ID = 'zeta-yen-319702'
REGION = 'us-central1'
BUCKET = 'gs://zeta-yen-319702-temp/'

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET
)

In [25]:
from llama_index.llms.vertex import Vertex
from llama_index.llms.base import ChatMessage, MessageRole, CompletionResponse

llm = Vertex(model="text-bison", temperature=0, additional_kwargs={})
llm.complete("Hello this is a sample text").text

' ```\nHello this is a sample text\n```'

In [29]:
chat = Vertex(model="chat-bison")
messages = [
    ChatMessage(role=MessageRole.SYSTEM, content="Reply everything in french"),
    ChatMessage(role=MessageRole.USER, content="Hello"),
]
chat.chat(messages=messages).message.content

' Bonjour! Comment vas-tu?'

In [11]:
from langchain.embeddings import TensorflowHubEmbeddings
from langchain.vectorstores import DeepLake
url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed_model = TensorflowHubEmbeddings(model_url=url)

In [30]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext

service_context = ServiceContext.from_defaults(
    llm=chat, embed_model=embed_model
)

dataset_path = "paul_graham_essay"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [31]:
query_engine = index.as_query_engine()

In [32]:

response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

 1) Join existing projects. 
2) Keep reading and talking to people. 
3) Focus on an application area. 
4) Develop a side hustle. 
